## Loan Costing Discussion
This example uses the cost formulae from the `temoa_rules.py` file in order to verify costs used for loan.  The Temoa dox for the loan cost explain the original formula fairly well.  The original pointer to the formula is from Kevin Hunter et. all paper in ~2013.  See equation (15) on the included document and description.  Of particular note, that document uses the loan lifetime in a few spots, and somewhere along the way the tech lifetime crept into the formula in the form of the `lifetime_process` variable, which seems like a mistake



### Key Questions:
1. What is `discount_rate`?  It looks and smells like the loan rate.  The dox are just wrong.
2. Is the inclusion of the process lifetime in the loan cost calculation a typo?  Seems it should be loan life across the board
3. Is the approach to undiscounted correct?
4. It is unclear what the plan is for "hurdle rate" -- and if any support from me is needed on that

### Ponderings...
- Note that in none of these approaches renders the "full cost" of a scenario visible because of the eclipsing effect of the end of the window in perfect foresight or the end of the shorter myopic windows.
- A corollary of that (that I stumbled into the other day during testing) is that it is impossible to compare costs when using different myopic view depths.  More clearly:  When I changed from a view depth of 1 to 2 (seeing further into the future), I expected cost to go down--as they should--but they went up, which is attributable to having more of the loan lives visible to the calculation.  This may/may not be important to the team.  It is just an observation.
- It wouldn't be too much work to enable a 3rd mode for these to calculate the "full cost" by making the windowing feature conditional on some flag.

### Desired outcome:
- Concurrence or edits to formulae
- About 4 test values that I can incorporate into unit tests that exercise the discount rate, GDR, and eclipsing effect of the period end, and discounted/undiscounted test values (similar to what is used below)

### Formulae
The below 2 formulae are the current implementations in the model that I have extracted out of the cost computation so that they can be used both by the model and in post-processing--basically single-sourcing the formula.  It was replicated in several areas before, which is an invitation for problems.

### Notes:  
#### "Undiscounted" cost
This was previously calculated in post-processing (not used at all in the model) was pretty wonky.  I have disregarded it and I think the below is a better place to **start** the discussion.  The intent of the fomula for loan_cost is to produce a undiscounted cost when GDR==0.
#### discount_rate
I have not renamed this, but this is central to the discussion.  It is applied as and probably should be renamed as the `loan_rate`.  TBD on how you want to handle "hurdle rate."


In [14]:
from pyomo.environ import Var, Expression

In [15]:
def loan_annualization_rate(loan_rate: float | None, loan_life: int | float) -> float:
    """
    This calculation is broken out specifically so that it can be used for param creation
    and separately to calculate loan costs rather than rely on fully-built model parameters
    :param loan_rate:  The loan rate
    :param loan_life:  The term (years) of the loan

    """
    if not loan_rate:
        # dev note:  this should not be needed as the LoanRate param has a default (see the definition)
        return 1.0 / loan_life
    annualized_rate = loan_rate / (1.0 - (1.0 + loan_rate) ** (-loan_life))
    return annualized_rate

In [16]:
def loan_cost(
    capacity: float | Var,
    invest_cost: float,
    loan_annualize: float,
    lifetime_loan_process: float | int,
    P_0: int,
    P_e: int,
    GDR: float,
    vintage: int,
) -> float | Expression:
    """
    function to calculate the loan cost.  It can be used with fixed values to produce a hard number or
    pyomo variables/params to make a pyomo Expression
    :param capacity: The capacity to use to calculate cost
    :param invest_cost: the cost/capacity
    :param loan_annualize: parameter
    :param lifetime_loan_process: lifetime of the loan
    :param P_0: the year to discount the costs back to
    :param P_e: the 'end year' or cutoff year for loan payments
    :param GDR: Global Discount Rate
    :param vintage: the base year of the loan
    :return: fixed number or pyomo expression based on input types
    """
    if GDR == 0:  # return the non-discounted result
        regular_payment = capacity * invest_cost * loan_annualize
        payments_made = min(lifetime_loan_process, P_e - vintage)
        return regular_payment * payments_made
    x = 1 + GDR  # a convenience
    res = (
        capacity
        * (
            invest_cost
            * loan_annualize
            * (
                lifetime_loan_process
                if not GDR
                else (x ** (P_0 - vintage + 1) * (1 - x ** (-lifetime_loan_process)) / GDR)
            )
        )
        * (
            (1 - x ** (-min(lifetime_loan_process, P_e - vintage)))
            / (1 - x ** (-lifetime_loan_process))
        )
    )
    return res

### Exemplar tech data
Consider an anonymous `tech` in a myopic run with periods @ 5 year increments [2020, 2050] and a myopic view of 1 period at a time.
In the myopic window 2030 -> 2035 a decision is made to build 100K capacity units:


In [17]:
capacity = 100_000 # units
cost_invest = 1    # $/unit of capacity
loan_life = 40
loan_rate = 0.08
GDR = 0.05
tech_lifetime = 50
base_year = 2020   # the "myopic base year" to which all prices are discounted
vintage = 2030     # the vintage of the new 'tech'
window_end = 2035  # last year in the myopic view

#### We need LoanAnnualize...
LoanAnnualize is a model parameter that is computed within the model using the discount rate specific to that process.  I have also "extracted the math" to a separate function that can is also dual-use (making the parameter, or externally producing a hard number)


In [18]:
loan_annualize = loan_annualization_rate(loan_rate=loan_rate, loan_life=loan_life)

In [19]:
print(f"Loan annualization rate: {loan_annualize:0.4f}")

In [20]:
cost = loan_cost(
    capacity=capacity,
    invest_cost=cost_invest,
    loan_annualize=loan_annualize,
    lifetime_loan_process=loan_life,
    P_0=base_year,
    P_e=window_end,
    GDR=GDR,
    vintage=vintage
)

In [21]:
print(f"Loan cost: ${cost:,.2f}")

#### And the Undiscounted ...

In [22]:
undiscounted_cost = loan_cost(
    capacity=capacity,
    invest_cost=cost_invest,
    loan_annualize=loan_annualize,
    lifetime_loan_process=loan_life,
    P_0=base_year,
    P_e=window_end,
    GDR=0,  # <-- override with a zero
    vintage=vintage
)

In [23]:
print(f"Undiscounted Loan cost: ${undiscounted_cost:,.2f}")

### Are these correct?
The end of the window is eclipsing most future payments.
If the problem were "perfect foresight" then...

In [24]:
window_end = 2050  # overwrite the value of the window end

cost = loan_cost(
    capacity=capacity,
    invest_cost=cost_invest,
    loan_annualize=loan_annualize,
    lifetime_loan_process=loan_life,
    P_0=base_year,
    P_e=window_end,
    GDR=GDR,
    vintage=vintage
)
print(f"Loan cost: ${cost:,.2f}")

### Or if the end of the window exposed the entire loan life...


In [25]:
window_end = 2100 # override to expose full loan length

cost = loan_cost(
    capacity=capacity,
    invest_cost=cost_invest,
    loan_annualize=loan_annualize,
    lifetime_loan_process=loan_life,
    P_0=base_year,
    P_e=window_end,
    GDR=GDR,
    vintage=vintage
)
print(f"Loan cost: ${cost:,.2f}")

In [26]:
undiscounted_cost = loan_cost(
    capacity=capacity,
    invest_cost=cost_invest,
    loan_annualize=loan_annualize,
    lifetime_loan_process=loan_life,
    P_0=base_year,
    P_e=window_end,
    GDR=0,  # <-- set to zero
    vintage=vintage
)
print(f"Undiscounted Loan cost: ${undiscounted_cost:,.2f}")